# **COMPARACIÓN DE MODELOS**

# Índice

1. [Comparación teórica](#comparación-teórica)

    1.0. [Random Forest vs. Gradient Boosting (XGBoost)](#random-forest-vs-gradient-boosting-xgboost)

    1.1 [Random Forest vs. Árboles de Decisión Individuales](#random-forest-vs-árboles-de-decisión-individuales)
    
2. [Comparación práctica](#comparación-práctica)

    2.0. [Paquetes](#paquetes)

    2.1. [Evaluación](#evaluación)

## Comparación teórica

### Random Forest vs. Gradient Boosting (XGBoost)

| Aspecto | Random Forest | XGBoost |
|---------|---------------|---------|
| Paralelización | Entrenamiento paralelo de árboles | Construcción secuencial de árboles |
| Velocidad | Generalmente más rápido | Puede ser más lento pero a menudo más preciso |
| Overfitting | Menos propenso debido a la aleatoriedad | Más propenso, requiere regularización |
| Ajuste de hiperparámetros | Menos sensible a la configuración | Más sensible, requiere ajuste cuidadoso |
| Interpretabilidad | Importancia de características | Importancia de características + ganancia |



### Random Forest vs. Árboles de Decisión Individuales



Random Forest supera a los árboles individuales en:
- Reducción de varianza y overfitting
- Robustez frente a datos ruidosos
- Precisión general
- Estabilidad (pequeños cambios en los datos no afectan significativamente al modelo)

La principal desventaja es la pérdida de interpretabilidad respecto a un árbol individual.

# Comparación práctica

## Paquetes

In [ ]:
# Comparar curvas ROC
from sklearn.metrics import roc_curve

fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)

plt.figure(figsize=(10, 8))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_score(y_test, y_pred_proba):.4f})')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {roc_auc_score(y_test, y_pred_proba_xgb):.4f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curvas ROC - Comparación de Modelos')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def recomendar_modelo(y_test, y_pred_rf, y_pred_proba_rf, y_pred_xgb, y_pred_proba_xgb):
    """
    Recomienda el mejor modelo basado en múltiples métricas entre XGBoost y Random Forest   
    """
    # Calcular métricas para Random Forest
    report_rf = classification_report(y_test, y_pred_rf, output_dict=True)
    auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
    
    # Calcular métricas para XGBoost
    report_xgb = classification_report(y_test, y_pred_xgb, output_dict=True)
    auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
    
    # Puntuar cada modelo (mayor puntuación es mejor)
    score_rf = (report_rf['1']['recall'] * 0.4 +  # Recall es importante para detectar fraudes
                report_rf['1']['precision'] * 0.3 +
                auc_rf * 0.3)
    
    score_xgb = (report_xgb['1']['recall'] * 0.4 +
                 report_xgb['1']['precision'] * 0.3 +
                 auc_xgb * 0.3)
    
    # Comparar y hacer recomendación
    if score_rf > score_xgb:
        recomendacion = "Random Forest"
        razon = f"Mejor balance entre recall ({report_rf['1']['recall']:.3f}) y precisión ({report_rf['1']['precision']:.3f})"
    else:
        recomendacion = "XGBoost"
        razon = f"Mejor balance entre recall ({report_xgb['1']['recall']:.3f}) y precisión ({report_xgb['1']['precision']:.3f})"
    
    print("=== RECOMENDACIÓN DE MODELO ===")
    print(f"Modelo recomendado: {recomendacion}")
    print(f"Razón: {razon}")
    print(f"Puntuación Random Forest: {score_rf:.4f}")
    print(f"Puntuación XGBoost: {score_xgb:.4f}")
    print("===============================")
    
    return recomendacion

# Usar la función de recomendación
recomendacion = recomendar_modelo(y_test, y_pred, y_pred_proba, y_pred_xgb, y_pred_proba_xgb)

c:\Users\agust\miniconda3\envs\mlparadetectarfraudes\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\agust\miniconda3\envs\mlparadetectarfraudes\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\agust\miniconda3\envs\mlparadetectarfraudes\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

=== RECOMENDACIÓN DE MODELO ===
Modelo recomendado: XGBoost
Razón: Mejor balance entre recall (0.233) y precisión (0.055)
Puntuación Random Forest: 0.1660
Puntuación XGBoost: 0.2779


## Evaluación

In [ ]:
# Define models to test
models = {
    'RandomForest': RandomForestClassifier(class_weight='balanced', random_state=42),
    'XGBoost': XGBClassifier(scale_pos_weight=(len(y)-sum(y))/sum(y), random_state=42),
    'LightGBM': LGBMClassifier(class_weight='balanced', random_state=42)
}

# Evaluation metrics
scoring = {
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'precision': 'precision',
    'recall': 'recall'
}

# Cross-validate each model
results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(sampling_strategy=0.3, random_state=42)),
        ('classifier', model)
    ])

    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    results[name] = {
        'f1_mean': np.mean(cv_results['test_f1']),
        'f1_std': np.std(cv_results['test_f1']),
        'roc_auc_mean': np.mean(cv_results['test_roc_auc']),
        'roc_auc_std': np.std(cv_results['test_roc_auc'])
    }

# Display results
results_df = pd.DataFrame(results).T
print(results_df)